In [1]:
import os
path = 'D:/python_project/hyunmin_project/운동동작분류/'

In [2]:
import glob
import pandas as pd
import numpy as np
import os 
from sklearn.model_selection import StratifiedKFold
import matplotlib.pylab as plt
from tqdm import tqdm
import re 
import tensorflow as tf
import keras
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, LearningRateScheduler
from sklearn.utils import shuffle
from tensorflow.keras.preprocessing.sequence import pad_sequences 
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from keras_self_attention import SeqSelfAttention
import random 
import time
from scipy import fftpack

train=pd.read_csv(path + 'open/train_features.csv')
train_label=pd.read_csv(path + 'open/train_labels.csv')
test=pd.read_csv(path + 'open/test_features.csv')
submission=pd.read_csv(path + 'open/sample_submission.csv')

In [31]:
X = tf.reshape(np.array(train.iloc[:,2:]), [-1,600,6]) 
X =  np.array(X).reshape(-1,600,6)
X.shape

(3125, 600, 6)

In [44]:
y = train_label['label'].values 

y =  tf.keras.utils.to_categorical(y)
y.shape

(3125, 61)

In [45]:
from attention import Attention
def build_model():  
    model= Sequential()
    model.add(Conv1D(64, input_shape=(600,6), kernel_size=3, padding='same',activation='relu'))
    model.add(Conv1D(64, kernel_size=3,padding='same',activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=3))
    model.add(Dropout(0.4))
    model.add(Conv1D(128,kernel_size=3,padding='same',activation='relu'))
    model.add(Conv1D(128,kernel_size=3,padding='same',activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=3))
    model.add(Dropout(0.4))
    model.add(Conv1D(256,kernel_size=3,padding='same',activation='relu'))
    model.add(Conv1D(256,kernel_size=3,padding='same',activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=3))
    model.add(Dropout(0.4))
    model.add(Conv1D(512,kernel_size=3,padding='same',activation='relu'))
    model.add(Conv1D(512,kernel_size=3,padding='same',activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=3))
    model.add(Dropout(0.4))
    #model.add(GlobalAveragePooling1D())
    #model.add(TimeDistributed(Flatten(), name='Flatten'))
    #model.add(Bidirectional(GRU(units=32, name='Bi-GRU')))
    #model.add(GRU(64,return_sequences=True))
    #model.add(Dropout(0.4))
    #model.add(keras.layers.Bidirectional(keras.layers.LSTM(units=128,
    #                                                   return_sequences=True)))
    #model.add(Flatten())
    #model.add(Attention())
    model.add(Dense(128,activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(64,activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(61, activation = 'softmax'))
    model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy']) 
    return model

In [46]:
def build_model2():  
    inputs = Input(shape = (600,6)) 
    bn = BatchNormalization()(inputs)
    bi_gru = Bidirectional(GRU(128, return_sequences = True))(bn) 
    dropout = Dropout(0.25)(bi_gru) 
    bi_gru = Bidirectional(GRU(128, return_sequences = False))(dropout) 
    dense = Dense(128, activation = 'relu')(bi_gru) 
    bn = BatchNormalization()(dense) 
    outputs = Dense(61, activation = 'softmax')(bn)   
    model = Model(inputs = inputs, outputs = outputs) 
    model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy']) 
    return model

In [56]:
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
mskf = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=42)
kfold = StratifiedKFold(n_splits = 5, random_state = 960418, shuffle = True)
for idx,(train_idx, val_idx) in enumerate(mskf.split(X,y)):
    print("... Validating on fold {} ...".format(idx+1)) 
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx] 
    
    ##### augment data #####
    print("... augmenting data ...")
    X_augmented = [] 
    y_augmented = [] 
    for i in tqdm(range(X_train.shape[0])): 
        for j in range(10): 
            shifted = np.roll(X_train[i], int(random.random() * 600)) 
            X_augmented.append(shifted) 
            y_augmented.append(y_train[i]) 
    X_augmented = np.array(X_augmented).reshape(-1,600,6) 
    y_augmented = np.array(y_augmented).reshape(-1,1)[:,0]
    y_augmented = tf.keras.utils.to_categorical(y_augmented)
    print(y_augmented.shape, y_train.shape)
    X_train = np.concatenate((X_train, X_augmented), axis=0) 
    y_train = np.concatenate((y_train, y_augmented), axis=0) 
    
    print(X_train.shape, y_train.shape, y_val.shape)
    
    ##### train model #####  
    print("... training ...") 
    model = build_model() 
    model_path = 'motion_detect/kfold' + str(idx+1) + '/epoch_{epoch:03d}_val_{val_loss:.3f}.h5'
    learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_loss', patience = 2, verbose = 1, factor = 0.8)
    checkpoint = ModelCheckpoint(filepath = model_path, monitor = 'val_loss', verbose = 1, save_best_only = True)
    early_stopping = EarlyStopping(monitor = 'val_loss', patience = 8) 

    model.fit(X_train,
              y_train,
              epochs=200,
              batch_size = 128,
              validation_data = (X_val, y_val),
              callbacks = [checkpoint, early_stopping])

C:\Users\gusals\anaconda3\envs\deep\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass shuffle=True, random_state=42 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


... Validating on fold 1 ...
... augmenting data ...


100%|█████████████████████████████████████████████████████████████████████████| 2500/2500 [00:00<00:00, 2935.23it/s]


(1525000, 2) (2500, 61)


ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 61 and the array at index 1 has size 2